# Analysis Overview

The goal of this notebook will be to provide an overview of the updated analysis of our study results and the choices that went into that anlaysis. A version of this notebook will be included in supplemental materials with directions on how to set up the requisite environment using [Anaconda](https://www.anaconda.com/). A version will also be hosted on [Github](https://github.com/) with an associated [Binder](https://mybinder.org/) envronment for improved acessibility.

*(n.b., this is simply DRAFT v1 -- current accompanying text is placeholder, needs cleaning up / fleshing out)*

---

## Intializiation

The following sections contain handle the requisite setup including the enivironment setup and data import.

In [1]:
# ---------- Imports ----------
%matplotlib widget

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import ipywidgets as widgets

from matplotlib.patches import Rectangle
from sklearn.neighbors import KernelDensity

import warnings
from copy import deepcopy

In [2]:
# ---------- Setup ----------
# data directory
dataDir = "./data/"

# color maps -- note convert from data specification to labels
cmap_label = ['Traditional', 'Jet', 'Kindlmann', 'Gray']
cmap_str = ['Default', 'Jet', 'Kindlmann', 'Gray']

# additional labels
param_label = ['L*', 'C*', 'h']
indicator_label = ['Cusps (High Curvature)', 'Inflection Points']

### Data Import

#### Color Maps
Here we load the four color maps used in our experiment.

In [3]:
# load all color maps, make accessible by label
cmaps = dict(zip(cmap_label,list(pd.read_csv(dataDir+v.lower()+".csv") for v in cmap_str)))

#### Participant Responses / Records
Here we load the particpant responses from the experiment and specify the outliers, as determined during the quantiative analysis found elsewhere in the supplemental materials.

In [4]:
# import records
records = pd.read_csv(dataDir+"categoryRecords.csv")
records2 = pd.read_csv(dataDir+"boundaryRecords.csv")

# outliers as dictated by our analysis
outliers = ['9229247b-f93a-4bd4-a47a-08ccc1addf2c', 
            '85434358-433b-4dcf-ac84-a87418eed8ef', 
            'c47869d4-cabe-40b9-89f4-74c76451dfe5', 
            '2652ec50-3918-492f-9d65-6210c2ca3031', 
            'cd9cf3f9-29f6-40df-baca-c3e77d71f0be', 
            'cfd3daa0-df19-4714-a47b-bf8d39aa74c5']

Given that our analysis does not appear to show marked differenes between our wording conditions, those records could be analyzed together. For those interested, a merged set of records can be created as follows.

### Convenience Definitions

Below are various function defintions used within the notebook.

In [5]:
# ---------- Convenience Functions ----------
from ConvenienceClasses import SingleLinePlotHandle, Interpolated, ParamControl

# generates figure that plots curves w/ indicators
def plot_curves(cmaps, order=None, indicators=None, legend=False):
    
    # if no labels provided, use keys from required dict
    cmap_labels = list(cmaps.keys())
    if order is not None:
        cmap_labels = [cmap_labels[i] for i in order]
    
    # setup figure
    fig = plt.figure(figsize=(9.5,3.7))
    fig.subplots_adjust(top=0.90, right=0.97, left=0.07, bottom=0.05)

    # setup gridspecs
    gs = gridspec.GridSpec(5, len(cmaps.keys()), height_ratios=[2,4,4,4,1])
    
    # convenience switches
    ylims_switch = {'L*':[0,100], 'C*':[0,140], 'h':[0,2*np.pi]}
    colors_switch = {'L*':['#0E6089','#5E50A3'], 'C*':['#0E6089', '#DB7723'], 'h':['#992B65', '#1C9B37']}

    # for each color map
    for i in range(len(cmaps.keys())):
        
        # grab colormap
        colormap = cmaps[cmap_labels[i]]
        
        # convenience switch
        colormap_switch = {'L*': colormap.L, 'C*':colormap.C, 'h':colormap.h}

        # create image
        rgb = colormap[['r','g','b']].values
        rgb.shape = (1,rgb.shape[0],rgb.shape[1])
        rgb_image = np.repeat(rgb, 50, axis=0)
        
        # plot image
        ax = plt.subplot(gs[0,i])
        ax.imshow(rgb_image, interpolation='nearest', aspect='auto', extent=[0.0,1.0,0.0,1.0])
        ax.get_yaxis().set_visible(False)
        ax.get_xaxis().set_visible(True)
        ax.tick_params(labelbottom=False)  
        ax.set_title(cmap_labels[i])

        # store axes
        ax_handles = []

        # setup plots
        #   Luminance
        ax = plt.subplot(gs[1,i])
        ax.plot(colormap.x, colormap_switch['L*'], color='#333333')
        ax.set_xlim([0,1])
        ax.set_ylim(ylims_switch['L*'])
        ax.tick_params(labelbottom=False)  
        ax_handles.append(ax)

        #   Chroma    
        ax = plt.subplot(gs[2,i])
        ax.plot(colormap.x, colormap_switch['C*'], color='#333333')
        ax.set_xlim([0,1])
        ax.set_ylim(ylims_switch['C*'])
        ax.tick_params(labelbottom=False)
        ax_handles.append(ax)

        #   Hue    
        ax = plt.subplot(gs[3,i])
        ax.plot(colormap.x, colormap_switch['h'], color='#333333')
        ax.set_xlim([0,1])
        ax.set_ylim(ylims_switch['h'])
        ax.set_yticks([0, np.pi/2, np.pi, 3*np.pi/2, 2*np.pi])
        ax.set_yticklabels(['$0$', r'$\frac{\pi}{2}$', r'$\pi$', r'$\frac{3\pi}{2}$', r'$2\pi$'])
        ax_handles.append(ax)
        
        # if indicators available
        if indicators is not None:
            # grab color map label
            c = cmap_labels[i]
            # for parameter axes
            for j in range(len(ax_handles)):
                p = param_label[j]
                ax = ax_handles[j]
                cc = colors_switch[p]
                
                # draw indicators
                for xc in indicators[c][p]['Cusps (High Curvature)']:
                    ax.axvline(x=xc, color=cc[0], linestyle='--', alpha=0.55)
                
                for xc in indicators[c][p]['Inflection Points']:
                    ax.axvline(x=xc, color=cc[1], linestyle='--', alpha=0.55)
                    
        # add labels to right hand side of plots
        if (i==0):
            ax_handles[0].set_ylabel(r'L$^{*}$', rotation='horizontal', fontsize=14, va='center', ha='right', labelpad=3)
            ax_handles[1].set_ylabel(r'C$^{*}$', rotation='horizontal', fontsize=14, va='center', ha='right', labelpad=3)
            ax_handles[2].set_ylabel('h',  rotation='horizontal', fontsize=14, va='center', ha='right', labelpad=13)
    
    return fig

#  indicator derivation plots
def plotParam(figure, colormap, param='L*', xs=np.linspace(0,1,10000)):
    # setup gridspecs
    gs = gridspec.GridSpec(4, 1, height_ratios=[1.5,4,4,4])
    
    # switch dicts
    colormap_switch = {'L*': colormap.L, 'C*':colormap.C, 'h':colormap.h}
    ylims_switch = {'L*':[0,100], 'C*':[0,140], 'h':[0,2*np.pi]}
    
    # generate spline
    current_param = Interpolated(colormap.x, colormap_switch[param], 0.5)
    
    # create image
    rgb = colormap[['r','g','b']].values
    rgb.shape = (1,rgb.shape[0],rgb.shape[1])
    rgb_image = np.repeat(rgb, 50, axis=0)
    
    axes = []
    # plot image
    ax = plt.subplot(gs[0,0])
    im_obj = ax.imshow(rgb_image, interpolation='nearest', aspect='auto')
    ax.get_yaxis().set_visible(False)
    ax.tick_params(labelbottom=False)
    axes.append(ax)
    
    # plot spline
    ax = plt.subplot(gs[1,0])
    l, = ax.plot(xs, current_param.spline(xs), color='C0')
    ax.set_xlim([0,1])
    ax.tick_params(labelbottom=False)
    ax.set_ylim(ylims_switch[param])
    if (param is 'h'):
        ax.set_yticks([0, np.pi/2, np.pi, 3*np.pi/2, 2*np.pi])
        ax.set_yticklabels(['$0$', r'$\frac{\pi}{2}$', r'$\pi$', r'$\frac{3\pi}{2}$', r'$2\pi$'])
    current_param.handles.append(SingleLinePlotHandle(ax, l))
    axes.append(ax)
    
    # plot |grad|
    ax = plt.subplot(gs[2,0])
    l, = ax.plot(xs, np.abs(current_param.grad(xs)), color='C0')
    ax.set_xlim([0,1])
    ax.tick_params(labelbottom=False)
    current_param.handles.append(SingleLinePlotHandle(ax, l))
    axes.append(ax)
    
    # plot |curv|
    ax = plt.subplot(gs[3,0])
    l, = ax.plot(xs, np.abs(current_param.curv(xs)), color='C0')
    ax.set_xlim([0,1])
    current_param.handles.append(SingleLinePlotHandle(ax, l))
    axes.append(ax)
    
    return (axes, current_param, im_obj)

# Sigma Sensitivity Analysis
def sigma_senstitivity(colormap, param_str, indicator_str, sigma_values=[], thresholds=[], tol=0.005):
        
    colormap_switch = {'L*': colormap.L, 'C*':colormap.C, 'h':colormap.h}
    current_param = Interpolated(colormap.x, colormap_switch[param_str], 0)

    indicator_locations = []
    for i in range(len(sigma_values)):
        current_param.sigma = sigma_values[i]
        if indicator_str == indicator_label[0]:
            _list = current_param.getHighCurv(thresholds[i])
        elif indicator_str == indicator_label[1]:
            _list = current_param.getInflectionPoints(thresholds[i])
        else:
            _list = []
        indicator_locations.append(_list)

    indicator_sets = []
    iterators = np.array([iter(l) for l in indicator_locations])
    current = np.array([next(i, np.NaN) for i in iterators])

    nan_mask = np.isnan(current)
    with warnings.catch_warnings():
        warnings.simplefilter("ignore") #ignore NaN runtime warning -- we're relying on propogating NaNs for stop condition
        while not all(nan_mask):
            mask = (current - min(current)) < tol    
            push = current.copy()
            push[~nan_mask & ~mask] = np.NaN
            indicator_sets.append(push)
            current[~nan_mask & mask] = np.array([next(i, np.NaN) for i in iterators[~nan_mask & mask]])
            nan_mask = np.isnan(current)

    df = pd.DataFrame(np.transpose(np.array(indicator_sets)))
    df.insert(0, 'sigma', sigma_values)
    df.insert(1, 'threshold', thresholds)
    
    return (df)

# intial draft of indicator formatter -- still functional
def getPredictors(cmap_str, param_str, indicators, est = 0.07):
    colors_switch = {'L*':['#0E6089','#5E50A3'], 'C*':['#0E6089', '#DB7723'], 'h':['#992B65', '#1C9B37']}
    output = []
    for label in indicator_label:
        output = output+[[[i],[est],[colors_switch[param_str][indicator_label.index(label)]]] for i in indicators[cmap_str][param_str][label]]
    return output

# more flexible indicator formatter
def getFormattedPredictors(spec_tuples, indicators, est = 0.07):
    colors_switch = {'L*':['#0E6089','#5E50A3'], 'C*':['#0E6089', '#DB7723'], 'h':['#992B65', '#1C9B37']}
    output = []
    for cmap_str,param_str,label in spec_tuples:
        output = output + [[[i],[est],[colors_switch[param_str][indicator_label.index(label)]]] for i in indicators[cmap_str][param_str][label]]
    return output

# placement plots w/ predictors
def plot_delimeters(records, dataset, cmap, condition, axes=None, predictor_set=[[[],[],[]]], r_sorted=True, two_sd=True, bandwidth=0.045, kernel='epanechnikov'):
    
    fig_h = 6.5
    fig_t = 0.91
    fig_b = 0.13
    
    if axes is None:
        fig = plt.figure(figsize=(3.5,fig_h))
        fig.subplots_adjust(top=fig_t, right=0.93, left=0.15, bottom=fig_b)
        gs = gridspec.GridSpec(2, 1, height_ratios=[4,2.5])
        ax = plt.subplot(gs[0,0])
        ax2 = plt.subplot(gs[1,0])
    else:
        ax = axes[0] # add error check for improper number of axes
        ax2 = axes[1]
    
    # -------------------- Data Processing -----------------------
    
    # Grab Delimeters
    gr = records[(records.cmap == cmap.lower()) & (records.data == dataset)]
    if (two_sd):
        delimeters = (gr[~gr['UUID'].isin(outliers)]).iloc[:,6:-1]
    else:
        delimeters = gr.iloc[:,6:-1]
    if r_sorted:
        delimeters['count'] = delimeters.count(axis=1)
        delimeters = delimeters.sort_values('count').drop(['count'], axis=1)
    
    # ----------------------- Plot ---------------------------
    cmap_str = 'Traditional' if (cmap == 'Default') else cmap
    ax.set_title(cmap_str+" - "+dataset)
    ax.set_ylabel('Participant ID')
    ax.set_xlim([0, 1.0])

    ax2.set_ylabel('Probability Density')
    ax2.set_xlabel('Normalized Data Value')
    ax2.set_xlim([0, 1.0])
    
    # draw delimeter placements
    R = delimeters.shape[0]
    ry = np.ones(delimeters.shape[1])
    for r in range(R):
        ax.scatter(np.clip(delimeters.values[r], 0, 1), ry*(r+1), color='black', marker="|", s=30, lw=2, zorder=3)
    
    if (two_sd): ax.set_ylim([0, 32]) #consistency hack
    
    # estimate and plot PDF
    # instantiate KDE
    kde = KernelDensity(bandwidth=bandwidth, kernel='epanechnikov')
    # list all delimeters (flatten and remove null values)
    d_list = delimeters.values.flatten()
    d_list = d_list[~np.isnan(d_list)]
    # fit the KDE model based on the delimeter list
    kde.fit(d_list[:, None])
    
    sample_x = np.linspace(0, 1, 1000)
    ax2.fill_between(sample_x, np.exp(kde.score_samples(sample_x[:, None])), color='#0E6089', alpha=0.4)
    ax2.set_ylim([0, ax2.get_ylim()[1]])
    
    ylims = ax.get_ylim()
    ylims2 = ax2.get_ylim()

    # draw indicators and calculate corresponding percentatges
    for p_set in predictor_set:
        if (len(p_set) > 0):
            # # calculate merged band
            # min_x = 1.1
            # max_x = -0.1

            for j in range(len(p_set[0])):
                # grab component indicators
                p = p_set[0][j]
                bin_width = p_set[1][j]
                cc = p_set[2][j]
                
                # #calculate merged band, cont.
                # min_x = min(min_x,(p - bin_width/2))
                # max_x = max(max_x,(p + bin_width/2))

                # draw component indicators
                ax.axvline(x=p, color=cc, linestyle='--', alpha=0.4, zorder=2)
                ax.add_patch(Rectangle((p - bin_width/2, ylims[0]), bin_width, ylims[1]-ylims[0], facecolor=cc, alpha=0.2, zorder=1))
                ax2.axvline(x=p, color=cc, linestyle='--', alpha=0.4, zorder=2)
                # ax2.add_patch(Rectangle((p - bin_width/2, ylims2[0]), bin_width, ylims2[1]-ylims2[0], facecolor=colors2[cc], alpha=0.2, zorder=1))
                
            # with warnings.catch_warnings():
            #     warnings.filterwarnings('error')
            #     try:
            #         # calculate stats
            #         band = delimeters.where((delimeters < max_x)&(delimeters > min_x))
            #         k = np.any(~np.isnan(band), axis=1)
            #         mn, sd, n = np.nanmean(band), np.nanstd(band), len(k)
            #         # draw error bar
            #         ax2.errorbar(mn, np.sum(k)/n, color='#0E0E0E', alpha=0.7, xerr=2*sd, fmt='o', markersize='5', zorder=2)
            #     except RuntimeWarning:
            #         pass                    
        
    # ---------------------- End --------------------------
    # display
    if axes is None: return (fig)

# getDelimeters()
#   returns a numpy array of all delimeters from record_set 
#   placed by particpants for a given dataset and colormap (cmap)
#   [optional argument two_sd drops records of outlier participants]
def getDelimeters(record_set, dataset, cmap, two_sd=True):
    
    # get delimeters
    gr = record_set[(record_set.cmap == cmap) & (record_set.data == dataset)]
    if (two_sd):
        delimeters = (gr[~gr['UUID'].isin(outliers)]).iloc[:,6:-1]
    else:
        delimeters = gr.iloc[:,6:-1]
    
    # flatten and remove null values (preserving participant unnecessary for density estimation)
    d2 = delimeters.values.flatten()
    d2 = d2[~np.isnan(d2)]
    
    return(d2)


In [6]:
# Define Indicators

# the original indicators
indicators_full = {
    'Traditional': {
        'L*': {
            'Cusps (High Curvature)': [0.25098,  0.74902],
            'Inflection Points': [0.214502]
        },
        'C*': {
            'Cusps (High Curvature)': [0.25098,  0.74902],
            'Inflection Points': [0.096852,  0.244349,  0.256319,  0.355024,  0.638564,  0.759411,  0.913586]
        },
        'h': {
            'Cusps (High Curvature)': [],
            'Inflection Points': [0.205089,  0.832291]
        }
    },
    'Jet': {
        'L*': {
            'Cusps (High Curvature)': [0.109804,  0.376471,  0.639216,  0.890196],
            'Inflection Points': [0.012758,  0.326049,  0.348109,  0.361094,  0.672307,  0.902486,  0.919723,  0.987242]
        },
        'C*': {
            'Cusps (High Curvature)': [0.109804,  0.376471,  0.639216,  0.890196],
            'Inflection Points': [0.007324,  0.01144,  0.012927,  0.099824,  0.102156,  0.221824,  0.369223,  0.383768,  0.419042,  0.646978,  0.647978,  0.651844,  0.816664,  0.992781]
        },
        'h': {
            'Cusps (High Curvature)': [0.341176,  0.376471,  0.639216,  0.658824],
            'Inflection Points': [0.347344,  0.652104,  0.728837]
        }
    },
    'Kindlmann': {
        'L*': {
            'Cusps (High Curvature)': [],
            'Inflection Points': []
        },
        'C*': {
            'Cusps (High Curvature)': [0.098039,  0.2,  0.298039 , 0.596078,  0.698039,  0.8,  0.811765],
            'Inflection Points': [0.11162,  0.146164,  0.309554,  0.474843,  0.80841]
        },
        'h': {
            'Cusps (High Curvature)': [0.007843,  0.094118,  0.113725,  0.2,  0.301961,  0.32549,  0.701961,  0.878431,  0.901961,  0.988235],
            'Inflection Points': [0.107047,  0.210065,  0.356393,  0.720725,  0.885858,  0.990418]
        }
    },
    'Gray': {
        'L*': {
            'Cusps (High Curvature)': [],
            'Inflection Points': []
        },
        'C*': {
            'Cusps (High Curvature)': [],
            'Inflection Points': []
        },
        'h': {
            'Cusps (High Curvature)': [],
            'Inflection Points': []
        }
    }
}

# reduced, removes numeric artifacts
indicators = deepcopy(indicators_full)
indicators['Traditional']['C*']['Inflection Points'] = [0.096852,  0.355024,  0.638564,  0.913586]
indicators['Jet']['L*']['Inflection Points'] = [0.326049,  0.348109,  0.672307,  0.919723]
indicators['Jet']['C*']['Inflection Points'] = [0.221824,  0.419042,  0.816664]
indicators['Jet']['h']['Inflection Points'] = [0.728837]
indicators['Kindlmann']['C*']['Cusps (High Curvature)'] = [0.098039,  0.2,  0.298039 , 0.596078,  0.698039,  0.8]
indicators['Kindlmann']['C*']['Inflection Points'] = [0.146164,  0.474843]
indicators['Kindlmann']['h']['Inflection Points'] = [0.356393]

#### Notebook Formatting

In [7]:
%%html
<style> table{float:left} </style>

---

## Indicator Defintions 

We'll start with an overview of our finalized indicator set. 

The high level overview description is basically that based on our H2 hypotheses (todo: summarize), we are interested in cusps and inflection points in the CIELCH luminance, chroma, and hue profiles.

The resulting indicator sets used in our analysis are shown below and include:

- **L*** / **C***  Cusps (*via high curvature*)
- **h**  Cusps (*via high curvature*)
- **L*** Inflection Points
- **C*** Inflection Points
- **h**  Inflection Points

In [8]:
# plot curves
fig_curves = plot_curves(cmaps, indicators=indicators)
fig_curves.show()
#fig_curves.savefig(outfile, dpi=150)

FigureCanvasNbAgg()

## Derivation

For consistency / in line with how color maps are often applied in practice, each of the color maps we tested were created as 256 value color lookup tables. For each of color value in these tables we take the corresponding CIELCH luminance, chroma, and hue value. We use these values to create cubic-spline interpolating polynomials that approximate the underlying CIELCH profiles. 

To find cusps, we look for locations of *high curvature* in the interpolating polynomials. We use the roots of the third derivative of these interpolating polynomials to locate the local maxima / minima in the second derivative, which represents curvature. Taking the absolute value of second derivive provides curvature magnitude, which we can theshold to generate a set of local maxima with arbitrarily high curvature magnitude.

The inflection points are derived similarly, only starting with the roots of the second derivative (i.e., the locations of zero curvature) and thresholding based on gradient magnitude.

Becuase numerical approximations are known to be sensitive to noise, we also include a smoothing parameter $\sigma$, which applies Gaussian smoothing to the luminance, chroma, and hue values from the color scale before creating the cubic-spline interpolating polynomials.

The specific smoothing and thesholding parameters we used can be all be found in the table below.

| Color Map   | Indicator              | Parameter | $\sigma$ | $\tau$ |
|------------:|:-----------------------|:---------|:--------|:------|
| Traditional |High Curvature         | L*        | 0       | 90000 |
| Traditional |High Curvature         | C*        | 0       | 37000 |
| Traditional |Inflection Points      | L*        | 0.6     | 295   |
| Traditional |Inflection Points      | C*        | 1.0     | 135   |
| Traditional |Inflection Points      | h         | 0       | 3     |
| Jet         |High Curvature         | L*        |       0 |  60000|
| Jet         |High Curvature         | C*        |       0 |  80000|
| Jet         |High Curvature         | h         |       0 |    990|
| Jet         |Inflection Points      | L*        |     1.2 |    158|
| Jet         |Inflection Points      | C*        |     0.8 |    100|
| Jet         |Inflection Points      | h         |     1.0 |      6|
| Kindlmann   |High Curvature         | C*        |     1.3 |  18000|
| Kindlmann   |High Curvature         | h         |     1.7 |    270|
| Kindlmann   |Inflection Points      | C*        |     1.5 |    370|
| Kindlmann   |Inflection Points      | h         |     2.5 |    3.4|

We've also included an interactive tool that illustrates the derivation process for each indicator set.

In [9]:
# ------------ INDICATOR DERIVATION TOOL ------------
from matplotlib.ticker import AutoLocator, ScalarFormatter

plt.ioff() # turns interactive mode off
plt.clf()  # clears current figure

# ------------ SMOOTHING / THRESHOLDING PARAMETERS ------------

# dictionary for switching settings
# returns (sigma, gradient_threshold, curvature_threshold)
indicator_settings = {
    'Traditional': {
        'L*': {
            'Cusps (High Curvature)': (0, None, 90000),
            'Inflection Points': (0.6, 295, None)
        },
        'C*': {
            'Cusps (High Curvature)': (0, None, 37000),
            'Inflection Points': (1.0, 135, None)
        },
        'h': {
            'Cusps (High Curvature)': (0, None, None),
            'Inflection Points': (0, 3, None)
        }
    },
    'Jet': {
        'L*': {
            'Cusps (High Curvature)': (0, None, 60000),
            'Inflection Points': (1.2, 158, None)
        },
        'C*': {
            'Cusps (High Curvature)': (0, None, 80000),
            'Inflection Points': (0.8, 100, None)
        },
        'h': {
            'Cusps (High Curvature)': (0, None, 990),
            'Inflection Points': (1.0, 6, None)
        }
    },
    'Kindlmann': {
        'L*': {
            'Cusps (High Curvature)': (0, None, None),
            'Inflection Points': (0, None, None)
        },
        'C*': {
            'Cusps (High Curvature)': (1.3, None, 18000),
            'Inflection Points': (1.5, 370, None)
        },
        'h': {
            'Cusps (High Curvature)': (1.7, None, 270),
            'Inflection Points': (2.5, 3.4, None)
        }
    },
    'Gray': {
        'L*': {
            'Cusps (High Curvature)': (0, None, None),
            'Inflection Points': (0, None, None)
        },
        'C*': {
            'Cusps (High Curvature)': (0, None, None),
            'Inflection Points': (0, None, None)
        },
        'h': {
            'Cusps (High Curvature)': (0, None, None),
            'Inflection Points': (0, None, None)
        }
    }
}

# ------------ INITIALIZE CONTROLS, PT. 1 ------------

# initialize controls
cmap_select = widgets.Select(
                                options=cmap_label,
                                value='Traditional',
                                rows=4,
                                description='Color Map:',
                                disabled=False
                             )

param_select = widgets.Select(
                                        options=param_label,
                                        value='L*',
                                        rows=3,
                                        description='Channel:',
                                        disabled=False
                             )

indicator_select = widgets.Select(
                                        options=indicator_label,
                                        value='Cusps (High Curvature)',
                                        rows=2,
                                        description='Indicators:',
                                        disabled=False
                             )

reset_button = widgets.Button(description='reset')

# ------------ SETUP FIGURE ------------

# setup figure
fig_edgePlot = plt.figure(figsize=(3.5,5))# 11,4
fig_edgePlot.subplots_adjust(top=0.95, right=0.95, left=0.17, bottom=0.05)

xs = np.linspace(0,1,10000) # values for plotting functions

# plot
axes, current_param, im_obj = plotParam(fig_edgePlot, cmaps[cmap_select.value], param_select.value, xs=xs)

# fig_edgePlot.show() will display a static plot, for interactive see below

# ------------ INITIALIZE CONTROLS, PT. 2 ------------

current_control = ParamControl(current_param, axes[1], axes[2], axes[3])

# ------------ UPDATE SCHEMES ------------
    
def update(figure, xs=[], control=None, grad=False, curv=False): 
        
    if control is not None: 
        if   grad: control.update_grad()
        elif curv: control.update_curv()
        else:      control.update_full(xs)
        
        figure.canvas.draw_idle()
        figure.canvas.flush_events()
        
def override(figure, axes, colormap, param_str, controls, im_obj=None):
    
    # override data
    param_switch = {'L*': colormap.L, 'C*':colormap.C, 'h':colormap.h}
    current_param.override(colormap.x, param_switch[param_str], current_param.sigma)
    
    # override ylims and ticks
    ylims_switch = {'L*':[0,100], 'C*':[0,140], 'h':[0,2*np.pi]}
    axes[1].set_ylim(ylims_switch[param_str])
    if (param_str == 'h'):
        axes[1].set_yticks([0, np.pi/2, np.pi, 3*np.pi/2, 2*np.pi])
        axes[1].set_yticklabels(['$0$', r'$\frac{\pi}{2}$', r'$\pi$', r'$\frac{3\pi}{2}$', r'$2\pi$'])
    else:
        axes[1].yaxis.set_major_locator(AutoLocator())
        axes[1].yaxis.set_major_formatter(ScalarFormatter())
        
    # override indicator colors
    colors_switch = {'L*':['#0E6089','#5E50A3'], 'C*':['#0E6089', '#DB7723'], 'h':['#992B65', '#1C9B37']}
    cc = colors_switch[param_str]
    controls.setColors(cc[0], cc[1])
    
    # re-draw
    update(figure, xs=xs, control=controls)

def update_cmap(change, im_obj):
    
    # create new image
    colormap = cmaps[cmap_select.value]
    
    # create new image
    rgb = colormap[['r','g','b']].values
    rgb.shape = (1,rgb.shape[0],rgb.shape[1])
    rgb_image = np.repeat(rgb, 50, axis=0)
    im_obj.set_data(rgb_image)
    
    # override plots
    override(fig_edgePlot, axes, colormap, param_select.value, current_control)
    
    #reset indicators
    update_indicators(current_control)  # indicator_select.value = None

def update_param(change):
    
    # override plots
    override(fig_edgePlot, axes, cmaps[cmap_select.value], param_select.value, current_control)
    
    # reset indicators
    update_indicators(current_control)  # indicator_select.value = None

def update_indicators(controls):
    
    s, g_thresh, c_thresh = indicator_settings[cmap_select.value][param_select.value][indicator_select.value]
    
    controls.sigma_control.value = s
    controls.gradient_threshold.value = g_thresh if g_thresh is not None else controls.gradient_threshold.max
    controls.curvature_threshold.value = c_thresh if c_thresh is not None else controls.curvature_threshold.max

# ------------ LINK CONTROLS ------------

current_control.sigma_control.observe(       lambda change : update(fig_edgePlot, control=current_control, xs=xs),     names='value' )
current_control.curvature_threshold.observe( lambda change : update(fig_edgePlot, control=current_control, curv=True), names='value' )
current_control.gradient_threshold.observe(  lambda change : update(fig_edgePlot, control=current_control, grad=True), names='value' )

cmap_select.observe( lambda change : update_cmap(change, im_obj), names='value')
param_select.observe(update_param, names='value')
indicator_select.observe(lambda change : update_indicators(current_control), names='value')

reset_button.on_click(lambda b : update_indicators(current_control))

# ------------ DISPLAY ------------

# update initial controls state
update_indicators(current_control)

# display
widgets.HBox([widgets.VBox([cmap_select,param_select,indicator_select,current_control.widget,reset_button]), fig_edgePlot.canvas])

### Sensitivity Analysis

I'll worry about expanding the writeup on this later, but this section will delve into:

1. Why we chose the smoothing and thesholding paramters that we did
2. Why we manually removed a subset of indicators

Looking at the sensitivity analysis we could get away with picking a single smoothing value for both the Traditional and Jet color maps. With the Kindlmann color map, I'd be a little more concerned due to the sheer amount of smoothing required to isolate the inflectction points in hue... As it is, though I think keeping the varying parmeters is fine as long as we're providing the justification like below.

- *(For readability we may want to reduce this to a few illustative examples and include a more extensive set of comparisons in a seperate notebook)*
- *(Also may want to plot some of these comparisons...)*

#### Traditional Rainbow

The high curvature indicators tend to be fairly invariant to a small amount of gaussian smoothing

In [10]:
sigma_senstitivity(cmaps['Traditional'], 'L*', 'Cusps (High Curvature)', sigma_values=[0, 0.6], thresholds=[60000, 60000])

,sigma,threshold,0,1
0,0.0,60000,0.25098,0.74902
1,0.6,60000,0.25098,0.74902


We can also see that the locations of high curvature in luminance and chroma are the same

In [11]:
sigma_senstitivity(cmaps['Traditional'], 'C*', 'Cusps (High Curvature)', sigma_values=[0, 0.8], thresholds=[30000,15000])

,sigma,threshold,0,1
0,0.0,30000,0.25098,0.74902
1,0.8,15000,0.25098,0.74902


The inflection point derivations on the other hand are not as inavraint to the gaussian smoothing...

In [12]:
sigma_senstitivity(cmaps['Traditional'], 'L*', 'Inflection Points', sigma_values=[0, 0.6, 0.8], thresholds=[295, 295, 295])

,sigma,threshold,0,1,2,3
0,0.0,295,0.214402,0.233036,0.235872,0.243943
1,0.6,295,0.214502,NaN,NaN,NaN
2,0.8,295,0.214556,NaN,NaN,NaN


That being said, it's clear some these indicators have significantly more varaince than others (e.g. 3, 4, and 7 below). As these indicators all approximate the loactions of high curvature, however, they wind being removed from our analysis.

Because we use interpolating polynomials to approximate the undelrying L*, C*, and h profiles, the cusps in those profile are essentially super high spatial frequency changes in concavity. This necessitates there being zero crossings in the gradient nearby, which would not ne there if we had modeled the profiles as a peicewise polynomial, for example. This makes them artifacts of the numerical processes we chose, not quantities of interest.

In [13]:
sigma_senstitivity(cmaps['Traditional'], 'C*', 'Inflection Points', sigma_values=[0, 0.8], thresholds=[135, 135])

,sigma,threshold,0,1,2,3,4,5,6,7,8
0,0.0,135,0.002037,0.008302,0.096767,0.247227,0.253740,0.355011,0.638593,0.75501,0.913557
1,0.8,135,NaN,NaN,0.096822,0.244724,0.255113,0.355019,0.638575,0.75749,0.913576


#### Jet

Again we see invariance in the high curvature indicators...

In [14]:
sigma_senstitivity(cmaps['Jet'], 'C*', 'Cusps (High Curvature)', sigma_values=[0, 0.6, 0.8, 1.0, 1.2], thresholds=[80000,50000,27000,27000,24000])

,sigma,threshold,0,1,2,3
0,0.0,80000,0.109804,0.376471,0.639216,0.890196
1,0.6,50000,0.109804,0.376471,0.639216,0.890196
2,0.8,27000,0.109804,0.376471,0.639216,0.890196
3,1.0,27000,0.109804,0.376471,0.639216,0.890196
4,1.2,24000,0.109804,0.376471,0.639216,0.890196


In [15]:
sigma_senstitivity(cmaps['Jet'], 'C*', 'Inflection Points', sigma_values=[0.65,0.8,1.0, 1.1], thresholds=[100,100,100,100])

,sigma,threshold,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0.65,100,0.007029,0.011348,0.013345,0.100383,0.102187,0.221806,0.369576,0.382988,0.419001,0.646387,0.648714,0.652351,0.816658,0.90163,0.902898,0.993096
1,0.80,100,0.007324,0.011440,0.012927,0.099824,0.102156,0.221824,0.369223,0.383768,0.419042,0.646978,0.647978,0.651844,0.816664,NaN,NaN,0.992781
2,1.00,100,0.010918,NaN,NaN,NaN,NaN,0.221855,0.368720,0.385938,0.419110,0.650843,NaN,NaN,0.816674,NaN,NaN,0.992168
3,1.10,100,0.011598,NaN,NaN,NaN,NaN,0.221873,0.368304,0.387276,0.419149,0.650844,NaN,NaN,0.816680,NaN,NaN,0.989277


With the potential L* Inflection Points (see 5/6, 10, 14 below) the varaiation due to smoothing is more pronounced

In [16]:
sigma_senstitivity(cmaps['Jet'], 'L*', 'Inflection Points', sigma_values=[0.8, 1.1], thresholds=[158,158], tol=0.004)

,sigma,threshold,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,0.8,158,0.007324,0.011440,0.012927,0.099826,0.102156,0.329803,0.346421,0.365157,0.648553,0.652901,0.669380,0.897623,0.900496,0.904003,0.916876,0.987073,0.98856,0.992676
1,1.1,158,NaN,0.011598,NaN,NaN,NaN,0.326636,0.347749,0.361603,NaN,NaN,0.670879,NaN,0.902313,NaN,0.918847,0.988402,NaN,NaN


In [17]:
sigma_senstitivity(cmaps['Jet'], 'L*', 'Cusps (High Curvature)', sigma_values=[0,1.1], thresholds=[60000,14000])

,sigma,threshold,0,1,2,3
0,0.0,60000,0.109804,0.376471,0.639216,0.890196
1,1.1,14000,0.109804,0.376471,0.639216,0.890196


#### Kindlmann

In [18]:
sigma_senstitivity(cmaps['Kindlmann'], 'h', 'Cusps (High Curvature)', sigma_values=[1.7,2.5], thresholds=[270,180])

,sigma,threshold,0,1,2,3,4,5,6,7,8,9,10,11
0,1.7,270,0.007843,0.094118,0.113725,0.200000,0.301961,0.32549,NaN,0.701961,0.878431,0.901961,NaN,0.988235
1,2.5,180,NaN,0.094118,0.117647,0.196078,0.301961,NaN,0.431373,0.701961,0.874510,0.901961,0.980392,NaN


(2) is the only non-confounded inflection point / the only one included in our analysis

In [19]:
sigma_senstitivity(cmaps['Kindlmann'], 'h', 'Inflection Points', sigma_values=[2.5], thresholds=[3.4])

,sigma,threshold,0,1,2,3,4,5
0,2.5,3.4,0.107047,0.210065,0.356393,0.720725,0.885858,0.990418


### Removing Artifacts

The indicator derivation procedures outlines above only get us so far. Based on our choices of smoothing and threshlding parameters we are left with indicator sets that look like:

In [20]:
plot_curves(cmaps, indicators=indicators_full).show()

FigureCanvasNbAgg()

As mentioned in the sensitivity analysis a number of these indicators can be traced to numeric artifacts. When these are removed we are left with the following, which is what we use in our anlysis.

*(needs significant expansion -- aside from spurious kindlamnn indicator, the discussion is laregly tied to the sensitivity anlaysis discussion, which is also incomplete)*

In [21]:
plot_curves(cmaps, indicators=indicators).show()

FigureCanvasNbAgg()

---
## Analysis: H1 Hypothesis

### Quantiative Analysis (Based on Counts)
We need a brief discussion of the fact that we did run analysis based on counts, and there were some significant data x colormap interactions. The scripts and detailed results are included elseshere in supplmental materials. This is beauce an analysis based on counts cannot really disprove the null hypothesis. We need to know if there are trends in *where* particpants place responses. We can, however, still use the results to help us understand what's going on.

![mean_plot](images/means.png)

*Need to update plot to stylistically fit with the rest of the figures; but fine as a placeholder*

Looking at the means we can see the data x color map interaction, where for both the Jet and Kindlmann color maps particpants percieved significantly more boundaries for 2D vs complex, whereas greyscale shows no difference between 2D and complex. 

We also see that the mean number of delimters placed in greyscale is around 5. This could be due to a couple of different factors, including:

- ill-defined nature of directions to find color boundaries when there is no 'hue'
- numeric artifacts in the color scale *(flesh out -- caused by discrete samples x gamma curve interaction)*

### Qualitative Analysis

Have interactive tool -- *(need to generate static version for figure)*

Based on color category literature expected amount of individual varaiation for color category boundaries range from 0.78 to 0.11. 

Compare estimates based on extremes 0.78/2 = 0.39 vs 0.11/2 = 0.55

In [22]:
# ------------ KDE BAND WIDTH TOOL ------------
colors = ['#0E6089', '#992B65', '#5E50A3','#DB7723', '#1C9B37']
colors2 = ['#0E6089', '#992B65', '#DB7723','#1C9B37']

# -------------------- Plot -----------------------

# cmap = 'default'
# predictor_set = H2_Empty
bandwidth = 0.045
kernel='epanechnikov'
two_sd=True

bandwidth = widgets.FloatSlider( orientation='vertical',
                                 description='$bandwidth$',
                                 value=0.045,
                                 min=0,
                                 max=0.15,
                                 step=0.005,
                                 continuous_update=True )

display_1D = widgets.Checkbox(
                                value=True,
                                description='1D',
                                disabled=False
                             )

display_2D = widgets.Checkbox(
                                value=True,
                                description='2D',
                                disabled=False
                             )

display_Cx = widgets.Checkbox(
                                value=True,
                                description='Complex',
                                disabled=False
                             )

checkboxes = [display_1D, display_2D, display_Cx]


cmap_select = widgets.Select(
                                options=['Default', 'Jet', 'Kindlmann', 'Gray'],
                                value='Default',
                                description='Color Map:',
                                disabled=False
                             )

plt.ioff()

# initalize figure
fig_KDE = plt.figure(figsize=(6.0,2.25)) # 3.5:2.5 -> 1:.714
fig_KDE.subplots_adjust(top=0.90, right=0.95, left=0.05, bottom=0.10)
gs = gridspec.GridSpec(1, 2)

ax = plt.subplot(gs[0,0])
ax.set_title("Category")
ax.set_xlim([0, 1.0])
ax.grid(color='#E0E0E0', linestyle='-', linewidth=1, axis='x')
ax.set_axisbelow(True)

ax2 = plt.subplot(gs[0,1])
ax2.set_title("Boundary")
ax2.set_xlim([0, 1.0])
ax2.grid(color='#E0E0E0', linestyle='-', linewidth=1, axis='x')
ax2.set_axisbelow(True)

# instantiate KDE
kde = KernelDensity(bandwidth=bandwidth.value, kernel=kernel)
sample_x = np.linspace(0, 1, 1000)

# for each dataset
datasets = ['1D', '2D', 'Complex']
poly_collections = [[],[]]
delimeter_sets = [[],[]]
lines = [[],[]]
for i in range(len(datasets)):
    
    # -------- Category --------
    condition_idx = 0
    # grab delimeters
    delimeters = getDelimeters(records, datasets[i], cmap_select.value.lower())
    delimeter_sets[condition_idx].append(delimeters)
    # fit the KDE model
    kde.fit(delimeters[:, None])
    
    # plot
    pc = ax.fill_between(sample_x, np.exp(kde.score_samples(sample_x[:, None])), color=colors[i], alpha=0.4)
    pc.set_visible(checkboxes[i].value)
    poly_collections[condition_idx].append(pc)
    l, = ax.plot(delimeters, np.full_like(delimeters, (-0.1*(i+1))-0.01), '|', color=colors[i], markeredgewidth=1)
    lines[condition_idx].append(l)
    
    # -------- Boundary --------
    condition_idx = 1
    delimeters = getDelimeters(records2, datasets[i], cmap_select.value.lower())
    delimeter_sets[condition_idx].append(delimeters)
    # fit the KDE model
    kde.fit(delimeters[:, None])
    
    # plot
    pc = ax2.fill_between(sample_x, np.exp(kde.score_samples(sample_x[:, None])), color=colors[i], alpha=0.4)
    pc.set_visible(checkboxes[i].value)
    poly_collections[condition_idx].append(pc)
    l, = ax2.plot(delimeters, np.full_like(delimeters, (-0.1*(i+1))-0.01), '|', color=colors[i], markeredgewidth=1)
    lines[condition_idx].append(l)

# match y_lim
ylim1 = ax.get_ylim()
ylim2 = ax2.get_ylim()
ymax = max(ylim1[1],ylim2[1])
ax.set_ylim([ylim1[0], ymax])
ax2.set_ylim([ylim2[0], ymax])
    
# # draw indicators
# for p_set in predictor_set:
#     if (len(p_set) > 0):

#         for j in range(len(p_set[0])):
#             # grab component indicators
#             p = p_set[0][j]
#             cc = p_set[2][j]
#             # draw component indicators
#             ax.axvline(x=p, color=colors2[cc], linestyle='--', alpha=0.4, zorder=2)


def update_checkbox(change):
    # update visibility
    poly_collections[0][checkboxes.index(change.owner)].set_visible(change.new)
    poly_collections[1][checkboxes.index(change.owner)].set_visible(change.new)
    # re-draw
    fig_KDE.canvas.draw_idle()
    fig_KDE.canvas.flush_events()

def update_cmap(change):
    # update delimeters + plot
    kde = KernelDensity(bandwidth=bandwidth.value, kernel=kernel)
    for i in range(len(datasets)):
        delimeter_sets[0][i] = getDelimeters(records,  datasets[i], cmap_select.value.lower())
        kde.fit(delimeter_sets[0][i][:, None])
        poly_collections[0][i].get_paths()[0].vertices[1:len(sample_x)+1,1] = np.exp(kde.score_samples(sample_x[:, None]))
        lines[0][i].set_xdata(delimeter_sets[0][i])
        lines[0][i].set_ydata(np.full_like(delimeter_sets[0][i], (-0.1*(i+1))-0.01))
        delimeter_sets[1][i] = getDelimeters(records2, datasets[i], cmap_select.value.lower())
        kde.fit(delimeter_sets[1][i][:, None])
        poly_collections[1][i].get_paths()[0].vertices[1:len(sample_x)+1,1] = np.exp(kde.score_samples(sample_x[:, None]))
        lines[1][i].set_xdata(delimeter_sets[1][i])
        lines[1][i].set_ydata(np.full_like(delimeter_sets[1][i], (-0.1*(i+1))-0.01))
    # re-draw
    fig_KDE.canvas.draw_idle()
    fig_KDE.canvas.flush_events()

def update_bandwidth(change):
    # update kde instantiation
    kde = KernelDensity(bandwidth=bandwidth.value, kernel=kernel)
    # update plot
    for i in range(len(datasets)):
        kde.fit(delimeter_sets[0][i][:, None])
        poly_collections[0][i].get_paths()[0].vertices[1:len(sample_x)+1,1] = np.exp(kde.score_samples(sample_x[:, None]))
        kde.fit(delimeter_sets[1][i][:, None])
        poly_collections[1][i].get_paths()[0].vertices[1:len(sample_x)+1,1] = np.exp(kde.score_samples(sample_x[:, None]))
    # re-draw
    fig_KDE.canvas.draw_idle()
    fig_KDE.canvas.flush_events()

bandwidth.observe(update_bandwidth, names='value')
display_1D.observe(update_checkbox, names='value')
display_2D.observe(update_checkbox, names='value')
display_Cx.observe(update_checkbox, names='value')
cmap_select.observe(update_cmap, names='value')

# fig.show()
widgets.HBox([widgets.VBox([cmap_select, display_1D, display_2D, display_Cx]), bandwidth, fig_KDE.canvas])

We see clear patterns in rainbow color scales, compared to grayscale

Do see some patterns in greyscale... *(need plots comparing against fit of artifacts from color map and potential data boundaries in complex)*

---
## Analysis: H2 Hypothesis


General pattern that High Curvature L*/C* pretty good indicators, though not always. Also some correspondence with inflection points in C*.

In [23]:
# hide
spec = [ ('Jet','C*','Cusps (High Curvature)'),
         ('Jet','C*','Inflection Points'),
       ]

fig_h = 6.5
fig_t = 0.91
fig_b = 0.13
cmap = 'Jet'
predictor_set = getFormattedPredictors(spec, indicators)
ylim_overwite = [0, 3.5]
r_sorted = False
bandwidth = 0.055 # 0.07

fig = plt.figure(figsize=(10.5,fig_h))
fig.subplots_adjust(top=fig_t, right=0.93, left=0.15, bottom=fig_b)
gs = gridspec.GridSpec(2, 3, height_ratios=[4,2.5])

axes = [plt.subplot(gs[0,0]), plt.subplot(gs[1,0])]
plot_delimeters(records2, "1D",     cmap, "Boundary", axes=axes, predictor_set=predictor_set, bandwidth=bandwidth, r_sorted=r_sorted, two_sd=True)
if ylim_overwite is not None: axes[1].set_ylim(ylim_overwite)

axes = [plt.subplot(gs[0,1]), plt.subplot(gs[1,1])]
plot_delimeters(records2, "2D",     cmap, "Boundary", axes=axes, predictor_set=predictor_set, bandwidth=bandwidth, r_sorted=r_sorted, two_sd=True)
for ax in axes: ax.set_ylabel("") # overwrite shared axis labels
if ylim_overwite is not None: axes[1].set_ylim(ylim_overwite)

axes = [plt.subplot(gs[0,2]), plt.subplot(gs[1,2])]
plot_delimeters(records2, "Complex", cmap, "Boundary", axes=axes, predictor_set=predictor_set, bandwidth=bandwidth, r_sorted=r_sorted, two_sd=True)
for ax in axes: ax.set_ylabel("") # overwrite shared axis labels
if ylim_overwite is not None: axes[1].set_ylim(ylim_overwite)


fig.show()

FigureCanvasNbAgg()

No real correspondence with inflection points in L* or h

In [24]:
# hide
spec = [ ('Jet','L*','Cusps (High Curvature)'),
         ('Jet','L*','Inflection Points'),
         ('Jet','h','Inflection Points') 
       ]

fig_h = 6.5
fig_t = 0.91
fig_b = 0.13
cmap = 'Jet'
predictor_set = getFormattedPredictors(spec, indicators)
ylim_overwite = [0, 3.5]
r_sorted = False
bandwidth = 0.055 # 0.07

fig = plt.figure(figsize=(10.5,fig_h))
fig.subplots_adjust(top=fig_t, right=0.93, left=0.15, bottom=fig_b)
gs = gridspec.GridSpec(2, 3, height_ratios=[4,2.5])

axes = [plt.subplot(gs[0,0]), plt.subplot(gs[1,0])]
plot_delimeters(records2, "1D",     cmap, "Boundary", axes=axes, predictor_set=predictor_set, bandwidth=bandwidth, r_sorted=r_sorted, two_sd=True)
if ylim_overwite is not None: axes[1].set_ylim(ylim_overwite)

axes = [plt.subplot(gs[0,1]), plt.subplot(gs[1,1])]
plot_delimeters(records2, "2D",     cmap, "Boundary", axes=axes, predictor_set=predictor_set, bandwidth=bandwidth, r_sorted=r_sorted, two_sd=True)
for ax in axes: ax.set_ylabel("") # overwrite shared axis labels
if ylim_overwite is not None: axes[1].set_ylim(ylim_overwite)

axes = [plt.subplot(gs[0,2]), plt.subplot(gs[1,2])]
plot_delimeters(records2, "Complex", cmap, "Boundary", axes=axes, predictor_set=predictor_set, bandwidth=bandwidth, r_sorted=r_sorted, two_sd=True)
for ax in axes: ax.set_ylabel("") # overwrite shared axis labels
if ylim_overwite is not None: axes[1].set_ylim(ylim_overwite)


fig.show()

FigureCanvasNbAgg()

High curvature in h mostly where it corresponds with L*/C*

In [25]:
#hide
spec = [ 
         ('Jet','L*','Cusps (High Curvature)'),
         ('Jet','h','Cusps (High Curvature)'),
       ]

fig_h = 6.5
fig_t = 0.91
fig_b = 0.13
cmap = 'Jet'
predictor_set = getFormattedPredictors(spec, indicators)
ylim_overwite = [0, 3.5]
r_sorted = False
bandwidth = 0.055 # 0.07

fig = plt.figure(figsize=(10.5,fig_h))
fig.subplots_adjust(top=fig_t, right=0.93, left=0.15, bottom=fig_b)
gs = gridspec.GridSpec(2, 3, height_ratios=[4,2.5])

axes = [plt.subplot(gs[0,0]), plt.subplot(gs[1,0])]
plot_delimeters(records2, "1D",     cmap, "Boundary", axes=axes, predictor_set=predictor_set, bandwidth=bandwidth, r_sorted=r_sorted, two_sd=True)
if ylim_overwite is not None: axes[1].set_ylim(ylim_overwite)

axes = [plt.subplot(gs[0,1]), plt.subplot(gs[1,1])]
plot_delimeters(records2, "2D",     cmap, "Boundary", axes=axes, predictor_set=predictor_set, bandwidth=bandwidth, r_sorted=r_sorted, two_sd=True)
for ax in axes: ax.set_ylabel("") # overwrite shared axis labels
if ylim_overwite is not None: axes[1].set_ylim(ylim_overwite)

axes = [plt.subplot(gs[0,2]), plt.subplot(gs[1,2])]
plot_delimeters(records2, "Complex", cmap, "Boundary", axes=axes, predictor_set=predictor_set, bandwidth=bandwidth, r_sorted=r_sorted, two_sd=True)
for ax in axes: ax.set_ylabel("") # overwrite shared axis labels
if ylim_overwite is not None: axes[1].set_ylim(ylim_overwite)


fig.show()

FigureCanvasNbAgg()

---
## Analysis: H3 Hypothesis